In [1]:
import numpy as np
import cv2

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import load_model

In [4]:
classifier = load_model('model.h5')

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
label_dict = {0:'WITHOUT MASK',1:'WITH MASK'}
color_dict = {0:(0,0,255),1:(0,255,0)}

In [6]:
def detect(frame):
    t = 0
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    for (x,y,w,h) in faces:
        
        rol_frame = frame[y:y+h,x:x+w]
        resized = cv2.resize(rol_frame,(224,224)) 
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,224,224,3)) 
        #reshaped = np.vstack([reshaped])
        result = classifier.predict(reshaped)

        label = result[0,0]
        if label > 0.5:
            t = 1 
        else:
            t = 0
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[t],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[t],-1)
        cv2.putText(frame,label_dict[t],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    return frame

In [7]:
video_capture = cv2.VideoCapture(0 + cv2.CAP_DSHOW)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH,640)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT,480)
while True:
    
    _, frame = video_capture.read()
    canvas = detect(frame)
    cv2.imshow('result',canvas)
    key = cv2.waitKey(1)
    if key == 27:
        break
  
video_capture.release()
cv2.destroyAllWindows()